## Setup

In [1]:
import json
import datetime
import pandas as pd
import googlemaps
from pandas_geojson import to_geojson
from ipyleaflet import Map, GeoJSON, FullScreenControl, Marker, MarkerCluster, AwesomeIcon
from ipywidgets import Layout, HTML

cfg = json.load(open('secure/config.json'))
gmaps = googlemaps.Client(key=cfg['google_maps_api_key'])

## Add markers and show schools on map

In [2]:
def add_school_map_marker(row):
    icon_type = {
        'Gymnasium': AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='blue',
            icon_color='black',
            spin=False
        ),
        'Freie Waldorfschule': AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='green',
            icon_color='black',
            spin=False
        )
    }
    icon_no_public_transport = AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='black',
            icon_color='white',
            spin=False
        )
    marker_icon = icon_type[row.Schultyp] if not(pd.isna(row.travel_duration_min)) else icon_no_public_transport
    marker = Marker(icon=marker_icon, location=(row.latitude, row.longitude), draggable=False, title=row.Name)
    message = HTML()
    message.value = f'<a href="https://{row.Homepage}">{row.Name}</a><p>travel time: {row.travel_duration_min} min</p>'
    marker.popup = message
    return marker

# Load previously saved data
schools_of_interest = pd.read_parquet('parquet/schools_gps_travel.parquet', engine="pyarrow")

# Add map markers
schools_of_interest[['marker']] = pd.DataFrame(
    schools_of_interest.apply(
        lambda row: add_school_map_marker(row), axis=1
    ).tolist(), index=schools_of_interest.index)

munich = (48.144227, 11.560515)
m = Map(center=munich, zoom=9, layout=Layout(width='100%', height='900px'))
m.add_control(FullScreenControl())

waldorf_cluster = MarkerCluster(
    markers=(schools_of_interest[schools_of_interest.Schultyp=='Freie Waldorfschule']['marker'].tolist())
)
m.add_layer(waldorf_cluster);
gymnasium_cluster = MarkerCluster(
    markers=(schools_of_interest[schools_of_interest.Schultyp=='Gymnasium']['marker'].tolist())
)
m.add_layer(gymnasium_cluster);

display(m)

Map(center=[48.144227, 11.560515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…